In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import pickle
from copy import deepcopy

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from datasets import *

from sklearn.metrics import ndcg_score

import matplotlib.pyplot as plt

In [2]:
train_data = Data_for_transformer_cnn('datas_full_10k_1.pkl', which = 0, for_cnn=True)
vali_data = Data_for_transformer_cnn('datas_full_10k_1.pkl', which = -1,for_cnn=True)

In [3]:
batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
vali_loader = DataLoader(vali_data, batch_size=batch_size, shuffle=True)

In [9]:
class CNN_model(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
         nn.Conv2d(1, 8, 8),
         nn.ReLU(),
         nn.Conv2d(8, 32, 8),
         nn.ReLU(),
         nn.Conv2d(32,64,8),
         nn.ReLU(),
         nn.Conv2d(64,64,8),
         nn.ReLU(),
         nn.Conv2d(64,32,8),
         nn.ReLU(),
         nn.Conv2d(32,16,8),
         nn.ReLU(),
         nn.Conv2d(16,8, 8),
         nn.ReLU(),
         nn.Flatten(),
         nn.Linear(144072,16000),
         nn.ReLU(),
         nn.Linear(16000,1024),
         nn.ReLU(),
         nn.Linear(1024,512),
         nn.ReLU(),
         nn.Linear(512,256)         
        )
        
    def forward(self,X):
        return self.model(X)

In [10]:
def loss(p,y):
    return -torch.mean(y * torch.log_softmax(p, dim = -1))

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = CNN_model().to(device)

kl_loss = nn.KLDivLoss()

opt_adam = torch.optim.AdamW(model.parameters(), lr=0.0001)

m_loss = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_adam, mode='min', patience=3, factor=0.1, verbose=True)

/home/aletovv/anaconda3/envs/v-env/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [12]:
model_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of trainable parameters: {model_total_params}')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

model

Number of trainable parameters: 2322791520


CNN_model(
  (model): Sequential(
    (0): Conv2d(1, 8, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 32, kernel_size=(8, 8), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 64, kernel_size=(8, 8), stride=(1, 1))
    (5): ReLU()
    (6): Conv2d(64, 64, kernel_size=(8, 8), stride=(1, 1))
    (7): ReLU()
    (8): Conv2d(64, 32, kernel_size=(8, 8), stride=(1, 1))
    (9): ReLU()
    (10): Conv2d(32, 16, kernel_size=(8, 8), stride=(1, 1))
    (11): ReLU()
    (12): Conv2d(16, 8, kernel_size=(8, 8), stride=(1, 1))
    (13): ReLU()
    (14): Flatten(start_dim=1, end_dim=-1)
    (15): Linear(in_features=144072, out_features=16000, bias=True)
    (16): ReLU()
    (17): Linear(in_features=16000, out_features=1024, bias=True)
    (18): ReLU()
    (19): Linear(in_features=1024, out_features=512, bias=True)
    (20): ReLU()
    (21): Linear(in_features=512, out_features=256, bias=True)
  )
)

In [13]:
def vali(vali_loader, model, k = [5,10,None]):
    model.eval()
    
    device = next(model.parameters()).device
    acc = np.zeros(3)
    length = len(vali_loader)
    
    for batch in vali_loader:
        X = batch[0].to(device).unsqueeze(1)
        y = batch[1].cpu().view(-1).numpy()
            
        preds = model(X.float()).view(-1).detach().cpu().numpy()
        for j,i in enumerate(k):
            try:
                acc[j] += ndcg_score([list(y)], [list(preds)], k = i)
            except:
                length -= 1

            
    return acc / length

def train(train_dataloader, test_dataloader, model, loss_fn, optimizer, sheluder, epochs, is_test = True, is_sheluder = False ):
    device = next(model.parameters()).device
    
    pred_probs = nn.Softmax(dim = -1)
    y_probs = nn.Softmax(dim = -1)
    
    train_loss = []
    metric = []
    
    
    
    for i in range(epochs):
        loss_epoch = 0
        
        model.train()
        for batch in train_dataloader:
            X = batch[0].float().to(device).unsqueeze(1)
            y = batch[1].squeeze().float().to(device)
            
            preds = model(X).squeeze()
            
            probas_p = preds
            probas_y = y
            
            optimizer.zero_grad()            
            loss_ = loss_fn(probas_p, probas_y)
            loss_.backward()
            optimizer.step()

            loss_epoch += loss_
          
        
        train_loss.append(loss_epoch.clone().detach().cpu().numpy()/len(train_dataloader))
        
        if is_test:
            model.eval()
            metric.append(vali(vali_loader=test_dataloader, model = model))
            print(f'loss for epoch {i+1} is {train_loss[i]:.4f} ||for epoch{i+1} ndcg@5_score is {metric[i][0]:.4f} || ndcg@10_score is {metric[i][1]:.4f} || ndcg_score is {metric[i][2]:.4f}  ')
        else:    
            print(f'loss for epoch {i+1} is {train_loss[i]:.4f}')
            
        if is_sheluder:   
            sheluder.step(loss_epoch)
            
    return np.array(train_loss), np.array(metric)
    
        
def plot_loss_and_metric(train_loss, name, is_metric = True, metric = []):
    size = len(train_loss)
    
    if is_metric:
        fig, ax = plt.subplots(1,4, figsize = (20,6), constrained_layout=True)
        
        
        ax[0].set_yscale('log')
        ax[0].plot(np.arange(size), train_loss)
        ax[0].set_xlabel('epochs')
        ax[0].set_ylabel('loss')
        ax[0].set_title('train loss')
        ax[0].grid(True)
        
        for i,j  in enumerate([5,10,'']):
        
            ax[i+1].plot(np.arange(size), metric[:,i])
            ax[i+1].set_xlabel('epochs')
            ax[i+1].set_ylabel(f'ndcg@{j}_score')
            ax[i+1].set_title('eval metric')
            ax[i+1].grid(True)

        
        fig.savefig(name)
        plt.show()
        
    else:
        fig, ax = plt.subplots(figsize = (6,6))
        fig.set_figheight(8)
        fig.set_figwidth(12)
        
        ax.set_yscale('log')
        ax.plot(np.arange(size), train_loss)
        ax.set_xlabel('epochs')
        ax.set_ylabel('loss')
        ax.set_title('train loss')
        ax.grid(True)
                
        fig.savefig(name)

        plt.show()

In [14]:
losses = train(train_dataloader=train_loader,
               test_dataloader= vali_loader,
      model = model,
      loss_fn=loss,
      optimizer=opt_adam,sheluder=scheduler,
      epochs=300)

OutOfMemoryError: CUDA out of memory. Tried to allocate 846.00 MiB. GPU 0 has a total capacity of 10.75 GiB of which 320.69 MiB is free. Including non-PyTorch memory, this process has 10.43 GiB memory in use. Of the allocated memory 10.23 GiB is allocated by PyTorch, and 12.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:
for i in train_loader:
    
    x = i[0].to(device).float().unsqueeze(1)
    print(x.shape)
    model(x)
    break

torch.Size([128, 1, 256, 136])
